# Stage 5 (3.4 recs_for_cold_users)


# Импортируем библиотеки

In [1]:
import warnings

import dill

import numpy as np
import pandas as pd


from rectools import Columns
from rectools.dataset import Dataset as RTDataset

from rectools.models import (
    PopularModel,
)
from mab2rec import BanditRecommender


from tqdm.auto import tqdm

from popular_bandit import PopularBanditRecommender
from metrics import RecommenderMetrics


warnings.filterwarnings("ignore")

RANDOM_STATE = 42

## Data paths

In [2]:
data_path = "../data_closed/"

In [3]:
models_path = "../models/"

In [4]:
candidates_data_path = models_path + "candidates_data/"

# Load Data & create Rectools Dataset

In [5]:
# Холодные пользователи из test_df, которым будут выданы
# рекомондации с помощью bandit recommender
with open(data_path + "test_only_users.dill", "rb") as f:
    test_only_users = dill.load(f)

test_only_users, test_only_users.shape

(array([4194307, 3145732, 4194309, ..., 1048568, 2097148, 4194301]), (542353,))

In [6]:
with (
    # Загружаем таблицу данных для моделей первого уровня
    open(data_path + "base_models_data.dill", "rb") as base_f,
    # Загружаем таблицу данных для ранкера
    open(data_path + "ranker_data.dill", "rb") as ranker_f,
):

    # # Сохраняем в отдельную переменную, т.к. нужно дообучить бандита
    mab_data = dill.load(ranker_f)

    # Создадим датасет взаимодействий для обучения TopPopular model
    current_dataset = RTDataset.construct(
        interactions_df=pd.concat(
            [
                dill.load(base_f)[["user_id", "item_id", "dt", "cum_weight"]],
                mab_data[["user_id", "item_id", "dt", "cum_weight"]],
            ],
            axis=0,
        ).rename(
            columns={
                "user_id": Columns.User,
                "item_id": Columns.Item,
                "dt": Columns.Datetime,
                "cum_weight": Columns.Weight,
            }
        )
    )

# BanditRecommender

## До

Бандит обучался следующим образом (ноутбук 1)

In [ ]:
# mab_model = BanditRecommender(
#     LearningPolicy.ThompsonSampling(),
#     top_k=10,
#     n_jobs=-1,
# )

In [ ]:
# # Загружаем таблицу данных для моделей первого уровня
# with open(data_path + "base_models_data.dill", "rb") as f:
#     base_models_data = dill.load(f)

In [ ]:
# mab_data = base_models_data[base_models_data["user_id"].isin(b2r_users)]
# mab_data = mab_data[(mab_data["u_total_inter"] > 20)]
# mab_data["binary_weight"] = (mab_data["ui_inter"] > 2).astype(int)
# mab_data = mab_data[
#     [
#         "user_id",
#         "item_id",
#         "dt",
#         "binary_weight",
#     ]
# ]
# mab_data

In [ ]:
# time_windows = []
# min_date = mab_data["dt"].min()
# max_date = mab_data["dt"].max()


# cur_min = min_date
# left, right = cur_min, cur_min + pd.Timedelta(hours=1.5)
# chunk = mab_data[mab_data["dt"].between(left, right, inclusive="left")]

# mab_model.fit(
#     decisions=chunk["item_id"],
#     rewards=chunk["binary_weight"],
# )

# print(f"Fitted: {left}, {right}")

# cur_min = right

# while right <= max_date:
#     left, right = cur_min, cur_min + pd.Timedelta(hours=1.5)
#     chunk = mab_data[mab_data["dt"].between(left, right, inclusive="left")]
#     mab_model.partial_fit(
#         decisions=chunk["item_id"],
#         rewards=chunk["binary_weight"],
#     )
#     cur_min = right
#     print(f"Fitted: {left}, {right}")


Fitted: 2024-10-10 00:00:01, 2024-10-10 01:30:01
Fitted: 2024-10-10 01:30:01, 2024-10-10 03:00:01
Fitted: 2024-10-10 03:00:01, 2024-10-10 04:30:01
Fitted: 2024-10-10 04:30:01, 2024-10-10 06:00:01
Fitted: 2024-10-10 06:00:01, 2024-10-10 07:30:01
Fitted: 2024-10-10 07:30:01, 2024-10-10 09:00:01
Fitted: 2024-10-10 09:00:01, 2024-10-10 10:30:01
Fitted: 2024-10-10 10:30:01, 2024-10-10 12:00:01
Fitted: 2024-10-10 12:00:01, 2024-10-10 13:30:01
Fitted: 2024-10-10 13:30:01, 2024-10-10 15:00:01
Fitted: 2024-10-10 15:00:01, 2024-10-10 16:30:01
Fitted: 2024-10-10 16:30:01, 2024-10-10 18:00:01
Fitted: 2024-10-10 18:00:01, 2024-10-10 19:30:01
Fitted: 2024-10-10 19:30:01, 2024-10-10 21:00:01
Fitted: 2024-10-10 21:00:01, 2024-10-10 22:30:01
Fitted: 2024-10-10 22:30:01, 2024-10-11 00:00:01
Fitted: 2024-10-11 00:00:01, 2024-10-11 01:30:01
Fitted: 2024-10-11 01:30:01, 2024-10-11 03:00:01
Fitted: 2024-10-11 03:00:01, 2024-10-11 04:30:01
Fitted: 2024-10-11 04:30:01, 2024-10-11 06:00:01
Fitted: 2024-10-11 0

## Теперь

Необходимо дообучить бандита на свежих данных (данные из ranker_data) 

In [7]:
with open(models_path + "mab_model.dill", "rb") as f:
    mab_model: BanditRecommender = dill.load(f)

In [ ]:
mab_data = mab_data[(mab_data["u_total_inter"] > 25)]
mab_data["binary_weight"] = (mab_data["ui_inter"] > 2).astype(int)
mab_data = mab_data[
    [
        "user_id",
        "item_id",
        "dt",
        "binary_weight",
    ]
]

In [18]:
min_date = mab_data["dt"].min()
max_date = mab_data["dt"].max()


cur_min = min_date
right = min_date

while right <= max_date:
    left, right = cur_min, cur_min + pd.Timedelta(hours=1.5)
    chunk = mab_data[mab_data["dt"].between(left, right, inclusive="left")][
        ["item_id", "binary_weight"]
    ]
    mab_model.partial_fit(
        decisions=chunk["item_id"],
        rewards=chunk["binary_weight"],
    )
    cur_min = right
    print(f"Fitted: {left}, {right}")

Fitted: 2024-10-11 07:59:58, 2024-10-11 09:29:58
Fitted: 2024-10-11 09:29:58, 2024-10-11 10:59:58
Fitted: 2024-10-11 10:59:58, 2024-10-11 12:29:58
Fitted: 2024-10-11 12:29:58, 2024-10-11 13:59:58
Fitted: 2024-10-11 13:59:58, 2024-10-11 15:29:58
Fitted: 2024-10-11 15:29:58, 2024-10-11 16:59:58


> Видимые накладки и несовпадения в выводе времени --- только результат реализации самого вывода, внутри все даты верные.

In [29]:
# Save model
with open(models_path + "mab_model.dill", "wb") as f:
    dill.dump(mab_model, f)

# Rectools PopularModel

In [30]:
# Train popular model on newer data
popular_model = PopularModel()

popular_model.fit(current_dataset)

# Save model
with open(models_path + "popular_model.dill", "wb") as f:
    dill.dump(popular_model, f)

# PopularBanditRecommender

In [7]:
knn_bandit_model = PopularBanditRecommender(
    dataset=current_dataset,
    path_bandit_model=models_path + "mab_model.dill",
    path_popular_model=models_path + "popular_model.dill",
    top_k=15,
)

## Predictions

In [8]:
candidates_mab = knn_bandit_model.predict(
    test_only_users,
    pop_k=50,
    pre_gen_recs=True,
    pre_gen_n=100,
)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
# Save candidates
with open(candidates_data_path + "candidates_mab.dill", "wb") as f:
    dill.dump(candidates_mab, f)

## Check metrics (even if predicitons are pretty random)

In [10]:
predictions = (
    candidates_mab[["user_id", "item_id"]]
    .groupby(by="user_id")["item_id"]
    .apply(list)
    .reset_index()
    .rename(columns={"item_id": "mab_recs"})
)

In [11]:
# Загружаем таблицу test_df
with open(data_path + "test_df.dill", "rb") as f:
    test_df = dill.load(f)


In [12]:
test_df = pd.merge(
    test_df[test_df["user_id"].isin(predictions["user_id"].unique())],
    predictions,
    how="left",
    on="user_id",
)

In [13]:
for k in [3, 5, 10, 15]:
    print(
        f"""k = {k}: {RecommenderMetrics.evaluate_recommender(
            test_df, model_preds_col="mab_recs", k=k
        )}\n"""
    )

k = 3: {'ndcg@k': 0.0, 'recall@k': 0.0, 'map@k': 0.0}

k = 5: {'ndcg@k': 0.0018428883314870617, 'recall@k': 0.0002127659574468085, 'map@k': 0.0005}

k = 10: {'ndcg@k': 0.0038976312448204408, 'recall@k': 0.0005404744436292492, 'map@k': 0.000575}

k = 15: {'ndcg@k': 0.004621466265233239, 'recall@k': 0.0009660063585228662, 'map@k': 0.000703968253968254}



k = 3: {'ndcg@k': 0.0, 'recall@k': 0.0, 'map@k': 0.0}

k = 5: {'ndcg@k': 0.0018428883314870617, 'recall@k': 0.0002127659574468085, 'map@k': 0.0005}

k = 10: {'ndcg@k': 0.0038976312448204408, 'recall@k': 0.0005404744436292492, 'map@k': 0.000575}

k = 15: {'ndcg@k': 0.004621466265233239, 'recall@k': 0.0009660063585228662, 'map@k': 0.000703968253968254}